In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score

import numpy as np

import keras
import keras.layers as layers
import tensorflow as tf
import os
from nltk.tokenize import word_tokenize
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout

from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [2]:
data = pd.read_csv('processed_data.csv')

In [3]:
data.head()

,title,text,class,text_without_stopwords,title_without_stopwords,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,...,polarity,overall_content,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability,polarity_category,polarity_category_Neutral,polarity_category_Positive
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,1,donald trump wish americans happy new year lea...,donald trump sends out embarrassing new year’s...,516,13,28,1,4.804040,...,0.082132,donald trump sends out embarrassing new year’s...,0.002194,0.747636,0.001007,0.157660,0.091503,Positive,0,1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,1,house intelligence committee chairman devin nu...,drunk bragging trump staffer started russian c...,309,9,11,1,5.213115,...,-0.005004,drunk bragging trump staffer started russian c...,0.064904,0.244962,0.557051,0.002320,0.130763,Neutral,1,0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",1,on friday revealed former milwaukee sheriff da...,sheriff david clarke becomes an internet joke ...,600,16,25,1,5.168966,...,-0.012345,sheriff david clarke becomes an internet joke ...,0.002488,0.433611,0.281460,0.001917,0.280524,Neutral,1,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",1,on christmas day donald trump announced would ...,trump is so obsessed he even has obama’s name ...,475,15,15,1,5.180180,...,-0.023118,trump is so obsessed he even has obama’s name ...,0.002963,0.788261,0.204377,0.002290,0.002109,Neutral,1,0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,1,pope francis used annual christmas day message...,pope francis just called out donald trump duri...,434,12,19,1,4.554762,...,-0.011722,pope francis just called out donald trump duri...,0.292172,0.327938,0.001138,0.020911,0.357842,Neutral,1,0


In [4]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
keys = contraction_dict.keys()

In [5]:
for i in range(len(data)):
    text = data['overall_content'].iloc[i]
    for word in keys:
        if word in data['overall_content'].iloc[i]:
            text = text.replace(word, contraction_dict[word])
        else:
            pass
    data.at[i, 'overall_content'] = text

In [7]:
x_train, x_test, y_train, y_test = train_test_split(data['overall_content'],data['class'],test_size=0.2,random_state = 4222)
print(x_train)

17157    hacking electronic voting machines easy 21st c...
5089     federal court strikes down north carolina’s ra...
7167     donald trump is staring down a criminal invest...
38280    explosions rock myanmar area near bangladesh b...
20108    trump's son close associates appear senate pre...
                               ...                        
22375    trump's pick lead us cftc unveils major new po...
34804    allies press catalan leader declare full indep...
31627    detained catalan government members say accept...
18325    us judge orders former trump aides stay home a...
1380     ‘lock him up’ passengers chant to disruptive m...
Name: overall_content, Length: 30926, dtype: object


In [8]:
max_features = 10000
maxlen = 500 # Set a max length of the array, if not it will do an array of like [1,10000] , and if i would to run the LSTM, it will take 30 hours

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train.squeeze())
tokenized_train = tokenizer.texts_to_sequences(x_train.squeeze())
x_train = pad_sequences(tokenized_train , maxlen=maxlen)

In [10]:
tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(tokenized_test , maxlen=maxlen)

In [12]:
max_features = 10000
batch_size = 256
epochs = 10
embed_size = 100

gru_model = keras.models.Sequential()
gru_model.add(layers.Embedding(max_features,embed_size))

gru_model.add(GRU(128, return_sequences=False))
gru_model.add(layers.Dropout(0.5))

gru_model.add(layers.Dense(32, activation='relu'))
gru_model.add(layers.Dropout(0.5))
gru_model.add(layers.Dense(1,activation="sigmoid",name="predictions"))

gru_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
print(gru_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 gru (GRU)                   (None, 128)               88320     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 predictions (Dense)         (None, 1)                 33        
                                                                 
Total params: 1,092,481
Trainable params: 1,092,481
Non-

In [13]:
gru_model.fit(x_train, y_train,validation_split=0.25,epochs=2)

Epoch 1/2
725/725 [==============================] - 365s 499ms/step - loss: 0.1450 - accuracy: 0.9505 - val_loss: 0.0476 - val_accuracy: 0.9841
Epoch 2/2
725/725 [==============================] - 352s 485ms/step - loss: 0.0397 - accuracy: 0.9876 - val_loss: 0.0565 - val_accuracy: 0.9815


In [ ]:
#overall_data = list(data['overall_content'])

#import gensim
#from gensim.utils import simple_preprocess

#def preprocess(text):
    #result = []
    #for article in text:
        #temporary = gensim.utils.simple_preprocess(article)
        #result.append(temporary)
    #return result

#all_words = preprocess(lst_x_train)

#lengths = [len(x) for x in all_words]
#max_lengths = max(lengths)
#print(max_lengths)

In [14]:
y_pred = (gru_model.predict(x_test) >= 0.5).astype("int")

242/242 [==============================] - 35s 144ms/step


In [15]:
import sklearn 
from sklearn.metrics import accuracy_score

In [16]:
print(accuracy_score(y_test, y_pred))

0.981246766683911
